In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp
import math as ma

In [2]:
# initial orbital state 
x=-9141.878
y=-1648.0758
z=4141.679
vx=-1.153
vy=-5.31
vz=-2.898

# final orbital state
a=12940
e=0.2173
i=0.8692
Om=1.448
w=2.721
f=2.827

mu=398600.433
R1=np.sqrt(x*x+y*y+z*z)


In [3]:
#Convert Keplerian elements to Cartesian.
def kep2cart(kep, GM):
    a,e,i,Om,w,f = kep
    p = a*(1-e*e)
    sqrt_GM = np.sqrt(GM/p)
    
    sini, cosi = np.sin(i), np.cos(i)
    sinOm, cosOm = np.sin(Om), np.cos(Om)
    sinw, cosw = np.sin(w), np. cos(w)
    sinf, cosf = np.sin(f), np.cos(f)
    
    rx = p*cosf/(1+e*cosf)
    ry = p*sinf/(1+e*cosf)
    v_x = -sqrt_GM*sinf
    v_y = sqrt_GM*(e+cosf)
    
    R11 = cosOm*cosw - sinOm*cosi*sinw
    R12 = -cosOm*sinw - cosw*cosi*sinOm
    R13 = sini*sinOm
    
    R21 = sinOm*cosw + cosOm*cosi*sinw
    R22 = -sinOm*sinw + cosOm*cosi*cosw
    R23 = -sini*cosOm
    
    R31 = sini*sinw
    R32 = sini*cosw
    R33 = cosi
    
    x = float(R11*rx + R12*ry)
    y = float(R21*rx + R22*ry)
    z = float(R31*rx + R32*ry)
    vx = float(R11*v_x + R12*v_y)
    vy = float(R21*v_x + R22*v_y)
    vz = float(R31*v_x + R32*v_y)
    
    return [x,y,z,vx,vy,vz]

In [4]:
#Convert Cartesian elements to Keplerian.
def cart2kep(cart, GM):
    x,y,z,vx,vy,vz = cart
    eps = 1e-14
    r = np.sqrt(x*x+y*y+z*z)
    rx = x
    ry = y 
    rz = z
    v = np.sqrt(vx*vx+vy*vy+vz*vz)
    E = (1/2)*v*v  - (mu/r)
    a = -mu/(2*E)
    hx = y*vz - z*vy
    hy = -x*vz + z*vx
    hz = x*vy - y*vx
    h = np.sqrt(hx*hx+hy*hy+hz*hz)
    i = np.arccos(hz/h)
    ex = (1/mu)*((v*v-mu/r)*rx - (rx*vx + ry*vy + rz*vz)*vx)
    ey = (1/mu)*((v*v-mu/r)*ry - (rx*vx + ry*vy + rz*vz)*vy)
    ez = (1/mu)*((v*v-mu/r)*rz - (rx*vx + ry*vy + rz*vz)*vz)
    e = np.sqrt(ex*ex+ey*ey+ez*ez)
    nx = -hy
    ny = hx
    nz = 0
    n = np.sqrt(nx*nx+ny*ny+nz*nz)
    
    if ny>=0:
        Om = np.arccos(nx/n)
        
    if ny<0:
        Om = 2*np.pi - np.arccos(nx/n)
    
    if e>eps:
        if n>eps:
             w = np.arccos((nx*ex+ny*ey)/n*e)
        if ez<0:
             w = 2*np.pi - np.arccos((nx*ex+ny*ey)/n*e)
   
    v_r = (x*vx+y*vy+z*vz)/r
    
    
    if v_r>=0:
        f = np.arccos((ex*rx+ey*ry+ez*rz)/(e*r))
                      
    if v_r<0:
        f = 2*np.pi - np.arccos((ex*rx+ey*ry+ez*rz)/(e*r))        
     
    
    return [a,e,i,Om,w,f]



In [5]:
IC = [ a , e, np.deg2rad(i), np.deg2rad(Om), np.deg2rad(w), f ];
ICCAR = kep2cart(IC,mu);


In [6]:
cart1=[x,y,z,vx,vy,vz]
C=cart2kep(cart1,mu)
a1=C[0]
e1=C[1]
w1=C[4]
f1=C[5]
Om1=C[3]
i1=C[2]
dOm=Om-Om1
di=i-i1
print("Om=",Om1)
print("a=",a1)
print("e=",e1)
print("i=",i1)
print("f=",f1)
print("w=",w1)
print("ΔΩ=",dOm)
print("Δi=",di)

T_per=2*np.pi*(np.sqrt(a1**3/mu))

Om= 0.7080148312799239
a= 9852.198421980704
e= 0.12072807884964891
i= 0.7230491866394563
f= 1.9566544240267003
w= 1.5581618269475754
ΔΩ= 0.739985168720076
Δi= 0.1461508133605437


In [7]:
a11=np.arccos(np.cos(i)*np.cos(i1)+np.sin(i)*np.sin(i1)*np.cos(dOm))

cosu2=(np.cos(i1)-np.cos(a11)*np.cos(i))/(np.sin(a11)*np.sin(i))
cosu1=(np.cos(i)-np.cos(a11)*np.cos(i1))/(np.sin(a11)*np.sin(i1))

sinu1=(np.sin(i)*(np.sin(dOm)/np.sin(a11)))
sinu2=(np.sin(i1)*(np.sin(dOm)/np.sin(a11)))
u1=np.arctan2(sinu1,cosu1)
u2=np.arctan2(sinu2,cosu2)
theta1=u1-w1
w2=u2-u1+w1
E1=np.arctan(np.sqrt((1-e1)/(1+e1))*np.tan(theta1/2))
M1=E1-e1*np.sin(E1)
n1=np.sqrt(mu/a1)
t1=M1/n1

if theta1>f1:
    dt1=t1
else:
    dt1=t1+T_per
        

In [9]:
dw=w-w2
theta2a=dw/2


E2=np.arctan(np.sqrt((1-e1)/(1+e1))*np.tan(theta2a/2))
M2=E2-e1*np.sin(E2)
n2=np.sqrt(mu/a1)
t2=M2/n2

if theta2a>theta1:
    dt2=t2-t1
else:
    dt2=t2-t1+T_per


In [10]:
theta3a=2*np.pi-theta2a
theta4=0
E3=np.arctan(np.sqrt((1-e1)/(1+e1))*np.tan(theta4/2))
M3=E3-e1*np.sin(E3)
n3=np.sqrt(mu/a1)
t3=M3/n3

if theta4>theta3a:
    dt3=t3-t2
else:
    dt3=t3-t2+T_per
    

In [11]:
E4=np.arctan(np.sqrt((1-e1)/(1+e1))*np.tan(f/2))
M4=E4-e1*np.sin(E4)
n4=np.sqrt(mu/a1)
t4=M4/n4

if f>np.pi:
    dt4=t4-t3
else:
    dt4=t4-t3+T_per
    

In [12]:
tof=dt1+dt2+dt3+dt4
print("The time-of-flight is:",tof)

The time-of-flight is: 29196.77922970034
